In [ ]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные модели
from sklearn import metrics #метрики
from sklearn import preprocessing # полиномиальные признаки
from sklearn.model_selection import train_test_split #сплитование выборки
%matplotlib inline
plt.style.use('seaborn-v0_8')

Этапы CRISP-DM
1. Анализ требований
2. Исследование данных
3. Подготовка данных
4. Моделирование
5. Оценка моедли

#### 1. Анализ требований
(ошибка модели +/- 2000 $)

In [ ]:
# загрузка данных
audi_data = pd.read_csv('data/audi.csv')
audi_data.head()

In [ ]:
# размерность таблицы
audi_data.shape

In [ ]:
# информация о дата-сете
audi_data.info()

# пропуски отсутствуют
# три категориальных признака

### Итерация №1

#### 2. Исследование данных

In [ ]:
# статистические данные
audi_data.describe()

In [ ]:
# статистические данные категориальных признаков
audi_data.describe(include='object')

In [ ]:
# создадим копию дата-сета
temp_data = audi_data.copy()

# создадим список числовых признаков
types = temp_data.dtypes
new_features = list(types[(types != 'object')].index)
new_features

Для каждого числового признака построим слева - гистограмма распрееления признака, справа - диаграмма рассеения с прямой "числовой регрессии", построинный с помощью МНК

In [ ]:
n = len(new_features)

fig, axes = plt.subplots(n, 2, figsize=(15,20))
for i, feature in enumerate (new_features):
    histplot = sns.histplot(data=temp_data, x=feature, ax=axes[i][0])
    histplot.set_title(f'{feature} hist')
    histplot.xaxis.set_tick_params(rotation=45)
    regplot = sns.regplot(data=temp_data, x=feature, y='price', ax=axes[i][1])
    regplot.set_title(f'{feature} VS price')
    regplot.xaxis.set_tick_params(rotation=45)
    
plt.tight_layout()

Выведем корреляцию признаков. Если корреляция хорошая, то и ленейная регрессия покажет себя также хорошо.

In [ ]:
#fig = plt.figure(figsize = (12, 5))
heatmap = sns.heatmap(temp_data.corr(), annot=True,)
display(heatmap)
#plt.show()

#### 2. Подготовка данных

In [ ]:
# горячее кодирование категориальных признаков
dummies_data = pd.get_dummies(temp_data)
dummies_data.head()

In [ ]:
# отделяем матрицу наблюдений и вектор правильных ответов
X = dummies_data.drop('price', axis=1)
y = dummies_data['price']

In [ ]:
# делим матрицу на тестовую и тренировочную выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)
print(f'Train shape: {X_train.shape}')
print(f'Test shape: {X_test.shape}')

#### 3. Моделирование

In [44]:
# функция для подсчета метрик R^2, MAE, MAPE для тренировочной и тестовой выборок

def print_metrics(y_train, y_train_predict, y_test, y_test_predict):
    print(f'Train^2: {metrics.r2_score(y_train, y_train_predict):.3f}')
    print(f'Train MAE: {metrics.mean_absolute_error(y_train, y_train_predict):.3f}')
    print(f'Train MAPE: {(metrics.mean_absolute_percentage_error(y_train, y_train_predict)*100):.3f}')
    print('\n')
    print(f'Test^2: {metrics.r2_score(y_test, y_test_predict):.3f}')
    print(f'Test MAE: {(metrics.mean_absolute_error(y_test, y_test_predict)):.3f}')
    print(f'Testt MAPE: {(metrics.mean_absolute_percentage_error(y_test, y_test_predict)*100):.3f}')

Модель линейной регрессии

Аналитическое решение: МНК

In [45]:
lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

print_metrics(y_train, y_train_pred, y_test, y_test_pred)

Train^2: 0.893
Train MAE: 2550.204
Train MAPE: 13.383


Test^2: 0.887
Test MAE: 2613.726
Testt MAPE: 12.876


Численное решение: метод градиентного спуска

In [47]:
# производим скалирование данных (стандартизация/нормализация)
scaler = preprocessing.MinMaxScaler() #стандартизация/нормализация
scaler.fit(X_train) # обучение на тренировочной выборке
X_train_scaled = scaler.transform(X_train) # трансформирование тренировочной выборки
X_test_scaled = scaler.transform(X_test) # трансформирование тестовой выборки

# подача трансформированных данных в модель SGD
sgd_lr = linear_model.SGDRegressor(random_state=42)
sgd_lr.fit(X_train_scaled, y_train)
y_train_pred = sgd_lr.predict(X_train_scaled)
y_test_pred = sgd_lr.predict(X_test_scaled)

print_metrics(y_train, y_train_pred, y_test, y_test_pred)

Train^2: 0.888
Train MAE: 2566.030
Train MAPE: 13.819


Test^2: 0.884
Test MAE: 2601.702
Testt MAPE: 12.876
